In [199]:
%pip install -U datasets ipywidgets jupyter Pillow ipykernel torchvision matplotlib

You should consider upgrading via the '/home/adam/.cache/pypoetry/virtualenvs/torchfactors-agsvsqrc-py3.8/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [200]:
from datasets import load_dataset
mnist = load_dataset('mnist')

2022-01-06 12:20:16,312 Reusing dataset mnist (/home/adam/.cache/huggingface/datasets/mnist/mnist/1.0.0/4bf7b6955107e5912c4fc84530a4284b4d5cec408474bd86185cfb7c81750003)


  0%|          | 0/2 [00:00<?, ?it/s]

In [243]:

from typing import Iterable
import torchfactors as tx
import torch
from dataclasses import dataclass

@dataclass
class MNISTExample(tx.Subject):
    pixels: tx.Var = tx.VarField(tx.Range(2), tx.OBSERVED)
    digit: tx.Var = tx.VarField(tx.Range(10), tx.ANNOTATED)
    cols = 28
    rows = 28
    
    @classmethod
    def from_lake(cls, example):
        return MNISTExample(
            pixels=tx.TensorVar(torch.tensor(
                # [[int(example['image'].getpixel((x,y)) > 150)
                [[float(example['image'].getpixel((x,y)))
                # [[int(example['image'].getpixel((x,y)) > 0)
                  for x in range(cls.cols)]
                 for y in range(cls.rows)])),
            digit=tx.TensorVar(torch.tensor(example['label']))
        )

train_raw = mnist['train']
n_full = len(train_raw)
n_train = 5000
n_dev = 1000
n_dev_splits = 5

index_ranges = dict(
    train=range(n_train),
    **{
        f'dev{i}': range(end - n_dev, end)
        for i, end in enumerate(range(n_full, n_train, -n_dev)[:n_dev_splits])
    })

subjects = {
    name: [MNISTExample.from_lake(train_raw[i]) for i in index_range]
    for name, index_range in index_ranges.items()
}

batches = {
    name: tx.Subject.stack(subs)
    for name, subs in subjects.items()
}


In [259]:
class CRF(tx.Model[MNISTExample]):
    def factors(self, x: MNISTExample) -> Iterable[tx.Factor]:
        yield tx.LinearFactor(self.namespace('digit'), x.digit, input=x.pixels.tensor)
        # yield tx.LinearFactor(self.namespace('digit'), x.digit, input=torch.cat([x.pixels.tensor, x.pixels.tensor, x.pixels.tensor], -1))
        intermediate = self.namespace('features1').module(tx.ShapedLinear, output_shape=(2,), input_shape=x.pixels.tensor.shape[1:])(x.pixels.tensor)
        intermediate = self.namespace('relu').module(torch.nn.ReLU)(intermediate)
        # intermediate = self.namespace('features2').module(tx.ShapedLinear, output_shape=(20,), input_shape=intermediate.shape[1:])(intermediate)
        # intermediate = self.namespace('relu').module(torch.nn.ReLU)(intermediate)
        intermediate = self.namespace('features4').module(tx.ShapedLinear, output_shape=(1,), input_shape=intermediate.shape[1:])(intermediate)
        intermediate = self.namespace('relu').module(torch.nn.ReLU)(intermediate)
        intermediate = self.namespace('features5').module(tx.ShapedLinear, output_shape=(2,), input_shape=intermediate.shape[1:])(intermediate)
        yield tx.LinearFactor(self.namespace('digit2'), x.digit, input=intermediate)
        # yield tx.LinearFactor(self.namespace('digit'), x.digit)
        # for r in range(x.rows):
        #     for c in range(x.cols):
        #         cell = x.pixels[...,r,c]
        #         # yield tx.LinearFactor(self.namespace(f'unigram_{r}_{c}'), cell, x.digit)
        #         yield tx.LinearFactor(self.namespace(f'position_{r}_{c}'), cell, input=x.digit.tensor)

system = tx.learning.example_fit_model(CRF(), subjects['train'], iterations=150, batch_size=n_train, lr=0.1, weight_decay=10.0)

2022-01-06 13:37:48,690 loading...
2022-01-06 13:37:48,692 done loading.
Subjects...: 100%|██████████| 1/1 [00:00<00:00,  6.07it/s]
2022-01-06 13:37:48,863 staring training...
100%|██████████| 150/150 [00:19<00:00,  7.78it/s, combo=2.66, i=149, j=0, loss=1.66, penalty=5.93e-8]       


In [260]:
predictions = {
    name: system.predict(batch)
    for name, batch in batches.items()
}
accuracies = {
    name: (batches[name].digit.tensor == predictions[name].digit.tensor).float().mean()
    for name in predictions
}
print(accuracies)



{'train': tensor(1.), 'dev0': tensor(0.9140), 'dev1': tensor(0.9270), 'dev2': tensor(0.8680), 'dev3': tensor(0.8680), 'dev4': tensor(0.8710)}


In [ ]:
# x = next(iter(MNISTExample.data_loader(train_loader, batch_size=n)))
# # x.pixels.set_usage(tx.LATENT)
# x.digit.set_usage(tx.LATENT)
# p_pixels = system.predict(x)
# from matplotlib import pyplot as plt
# # print(x.digit.tensor)
# # print(p_pixels.digit.tensor)
# # for i, xi in enumerate(p_pixels.unstack()):
# #     print(xi.digit.tensor)
#     # plt.figure(i)
#     # plt.imshow(xi.pixels.tensor.numpy())
# # plt.imshow(p_pixels.pixels.tensor.numpy())

In [ ]:
# # test_data = mnist['test']
# # n = len(test_data)
# # test_loader = [MNISTExample.from_lake(test_data[i]) for i in range(n)]
# test_loader = dev_loader
# test_batch = next(iter(MNISTExample.data_loader(test_loader, batch_size=n)))